In [32]:
#Problem (case study)
# We have data related to customers of an insurance company
# Am going to build and teach a Machine Learning model to predict Total claim amount for various customers 
# I will then test and validate my model

In [2]:
import pandas as pd
import numpy as np
import math
from sklearn import linear_model

In [3]:
# 1. Getting Data
data = pd.read_csv("marketing_customer_analysis.csv")
data.head()

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [4]:
data.columns

Index(['Customer', 'State', 'Customer Lifetime Value', 'Response', 'Coverage',
       'Education', 'Effective To Date', 'EmploymentStatus', 'Gender',
       'Income', 'Location Code', 'Marital Status', 'Monthly Premium Auto',
       'Months Since Last Claim', 'Months Since Policy Inception',
       'Number of Open Complaints', 'Number of Policies', 'Policy Type',
       'Policy', 'Renew Offer Type', 'Sales Channel', 'Total Claim Amount',
       'Vehicle Class', 'Vehicle Size'],
      dtype='object')

In [5]:
# Cleaning/Wrangling/EDA
data.dtypes

Customer                          object
State                             object
Customer Lifetime Value          float64
Response                          object
Coverage                          object
Education                         object
Effective To Date                 object
EmploymentStatus                  object
Gender                            object
Income                             int64
Location Code                     object
Marital Status                    object
Monthly Premium Auto               int64
Months Since Last Claim            int64
Months Since Policy Inception      int64
Number of Open Complaints          int64
Number of Policies                 int64
Policy Type                       object
Policy                            object
Renew Offer Type                  object
Sales Channel                     object
Total Claim Amount               float64
Vehicle Class                     object
Vehicle Size                      object
dtype: object

In [6]:
#CHANGE THEM TO LOWER CASE
new_cols = []
for cols in data.columns:
    new_cols.append(cols.lower())
data.columns = new_cols
data.columns

Index(['customer', 'state', 'customer lifetime value', 'response', 'coverage',
       'education', 'effective to date', 'employmentstatus', 'gender',
       'income', 'location code', 'marital status', 'monthly premium auto',
       'months since last claim', 'months since policy inception',
       'number of open complaints', 'number of policies', 'policy type',
       'policy', 'renew offer type', 'sales channel', 'total claim amount',
       'vehicle class', 'vehicle size'],
      dtype='object')

In [7]:
#RENAME SOME COLUMNS
data = data.rename(columns = {'sales channel':'channel','months since last claim':'last_claim'})
data.columns

Index(['customer', 'state', 'customer lifetime value', 'response', 'coverage',
       'education', 'effective to date', 'employmentstatus', 'gender',
       'income', 'location code', 'marital status', 'monthly premium auto',
       'last_claim', 'months since policy inception',
       'number of open complaints', 'number of policies', 'policy type',
       'policy', 'renew offer type', 'channel', 'total claim amount',
       'vehicle class', 'vehicle size'],
      dtype='object')

In [8]:
#CHECK FOR NAN
data.isna().sum()

customer                         0
state                            0
customer lifetime value          0
response                         0
coverage                         0
education                        0
effective to date                0
employmentstatus                 0
gender                           0
income                           0
location code                    0
marital status                   0
monthly premium auto             0
last_claim                       0
months since policy inception    0
number of open complaints        0
number of policies               0
policy type                      0
policy                           0
renew offer type                 0
channel                          0
total claim amount               0
vehicle class                    0
vehicle size                     0
dtype: int64

In [9]:
#THERE ARE NO NaN VALUES IN THE DATAFRAME

In [10]:
#NUMERICAL FEATURES
numerical = data.select_dtypes(include = np.number)
numerical.head()

,customer lifetime value,income,monthly premium auto,last_claim,months since policy inception,number of open complaints,number of policies,total claim amount
0,2763.519279,56274,69,32,5,0,1,384.811147
1,6979.535903,0,94,13,42,0,8,1131.464935
2,12887.431650,48767,108,18,38,0,2,566.472247
3,7645.861827,0,106,18,65,0,7,529.881344
4,2813.692575,43836,73,12,44,0,1,138.130879


In [11]:
categorical = data.select_dtypes(exclude = np.number)
categorical.head()

,customer,state,response,coverage,education,effective to date,employmentstatus,gender,location code,marital status,policy type,policy,renew offer type,channel,vehicle class,vehicle size
0,BU79786,Washington,No,Basic,Bachelor,2/24/11,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
1,QZ44356,Arizona,No,Extended,Bachelor,1/31/11,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize
2,AI49188,Nevada,No,Premium,Bachelor,2/19/11,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize
3,WW63253,California,No,Basic,Bachelor,1/20/11,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV,Medsize
4,HB64268,Washington,No,Basic,Bachelor,2/3/11,Employed,M,Rural,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car,Medsize


In [12]:
#CHECK UNIQUE VALUES FOR EACH COLUMN
for colx in categorical.columns:
    print(f"UNIQUE VALUES IN {colx} ARE: {categorical[colx].value_counts()}")


UNIQUE VALUES IN customer ARE: BU79786    1
PU81096    1
CO75086    1
WW52683    1
XO38850    1
          ..
HS14476    1
YL91587    1
CT18212    1
EW35231    1
Y167826    1
Name: customer, Length: 9134, dtype: int64
UNIQUE VALUES IN state ARE: California    3150
Oregon        2601
Arizona       1703
Nevada         882
Washington     798
Name: state, dtype: int64
UNIQUE VALUES IN response ARE: No     7826
Yes    1308
Name: response, dtype: int64
UNIQUE VALUES IN coverage ARE: Basic       5568
Extended    2742
Premium      824
Name: coverage, dtype: int64
UNIQUE VALUES IN education ARE: Bachelor                2748
College                 2681
High School or Below    2622
Master                   741
Doctor                   342
Name: education, dtype: int64
UNIQUE VALUES IN effective to date ARE: 1/10/11    195
1/27/11    194
2/14/11    186
1/26/11    181
1/17/11    180
1/19/11    179
1/31/11    178
1/3/11     178
1/20/11    173
2/26/11    169
1/28/11    169
2/19/11    168
1/5/11     1

In [13]:
#DROP CUSTOMER, effective to date, 
categorical = categorical.drop(['effective to date','customer'], axis = 1)
categorical.head()

,state,response,coverage,education,employmentstatus,gender,location code,marital status,policy type,policy,renew offer type,channel,vehicle class,vehicle size
0,Washington,No,Basic,Bachelor,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
1,Arizona,No,Extended,Bachelor,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize
2,Nevada,No,Premium,Bachelor,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize
3,California,No,Basic,Bachelor,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV,Medsize
4,Washington,No,Basic,Bachelor,Employed,M,Rural,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car,Medsize


In [14]:
#4. DATA PROCESSING

#DEALING WITH OUTLIERS. USING THE INCOME COLUMN AS THE REFERENCE
# iqr = np.percentile(numerical['income'],75) - np.percentile(numerical['income'],25)
# print(iqr)
iqr = np.quantile(numerical['income'],0.75) - np.quantile(numerical['income'],0.25)
print(iqr)
upper_limit = np.quantile(numerical['income'],0.75) + 1.5*iqr
lower_limit = np.quantile(numerical['income'],0.25) - 1.5*iqr
print(upper_limit)
print(lower_limit)
print(numerical.shape)

62320.0
155800.0
-93480.0
(9134, 8)


In [15]:
numerical = numerical[(numerical['income']>lower_limit) & (numerical['income']<upper_limit)]
#data = data[(data['IC1']>lower_limit) & (data['IC1']<upper_limit)]
print(numerical.shape)

(9134, 8)


In [16]:
# THERE ARE NO OUTLIERS IN THE DATA WHILE USING THE INCOME COLUMN
#IF THERE WERE, WE WOULD ADD THE CUSTOMER COLUMN TO THE NUMERICAL DATAFRAME BECAUSE CUSTOMER ID IS THE UNIQUE IDENTIFIER
#WE WOULD REMOVE ALL THE OUTLIERS FROM THE NUMERICAL DATAFRAME
#THEN JOIN/MERGE THE TWO DATAFRAME USING THE CUSTOMER ID. SINCE ITS THE UNIQUE IDENTIFIER
# THE MERGE METHOD WOULD REMOVE ALL CUSTOMERS FROM THE CATEGORICAL DATAFRAME WHOSE IDs ARE NOT INCLUDED IN THE NUMERICAL DATAFRAME

In [17]:
#NORMALIZE THE NUMERICAL DATA USING MINMAXSCALER
from sklearn.preprocessing import MinMaxScaler
kimera = MinMaxScaler().fit(numerical)
x_normalized = kimera.transform(numerical)
print(x_normalized.shape)
x_normalized = pd.DataFrame(x_normalized,columns=numerical.columns)
x_normalized.head()

(9134, 8)


,customer lifetime value,income,monthly premium auto,last_claim,months since policy inception,number of open complaints,number of policies,total claim amount
0,0.010629,0.562847,0.033755,0.914286,0.050505,0.0,0.000,0.132974
1,0.062406,0.000000,0.139241,0.371429,0.424242,0.0,0.875,0.391051
2,0.134960,0.487763,0.198312,0.514286,0.383838,0.0,0.125,0.195764
3,0.070589,0.000000,0.189873,0.514286,0.656566,0.0,0.750,0.183117
4,0.011245,0.438443,0.050633,0.342857,0.444444,0.0,0.000,0.047710


In [18]:
#ENCODING CATEGORICAL COLUMN
# AM LEAVING OUT STATE & LOCATION CODE. I BELIEVE THEY HAVE LIMITED IMPACT ON OUR LABEL
categorical.columns

Index(['state', 'response', 'coverage', 'education', 'employmentstatus',
       'gender', 'location code', 'marital status', 'policy type', 'policy',
       'renew offer type', 'channel', 'vehicle class', 'vehicle size'],
      dtype='object')

In [19]:
# categorical = categorical.drop(['state','location code'], axis = 1)
# categorical.columns

In [20]:
# ENCODING CATEGORICAL DATA USING ONE ENCODER
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder().fit(categorical)

#print(encoder.categories) - DOES NOT WORK. WHY???
encoded = encoder.transform(categorical).toarray()

# encoder.categories_ IS A NESTED ARRAY
#WE CAN FLATTEN AN ARRAY OF ARRAYS USING LISTS COMPREHESION
#Link: https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-a-list-of-lists
final_cols = [item for sublist in encoder.categories_ for item in sublist]
print(final_cols)
encoded = encoder.transform(categorical).toarray()
onehot_encoded = pd.DataFrame(encoded,columns=final_cols)
onehot_encoded.head()

['Arizona', 'California', 'Nevada', 'Oregon', 'Washington', 'No', 'Yes', 'Basic', 'Extended', 'Premium', 'Bachelor', 'College', 'Doctor', 'High School or Below', 'Master', 'Disabled', 'Employed', 'Medical Leave', 'Retired', 'Unemployed', 'F', 'M', 'Rural', 'Suburban', 'Urban', 'Divorced', 'Married', 'Single', 'Corporate Auto', 'Personal Auto', 'Special Auto', 'Corporate L1', 'Corporate L2', 'Corporate L3', 'Personal L1', 'Personal L2', 'Personal L3', 'Special L1', 'Special L2', 'Special L3', 'Offer1', 'Offer2', 'Offer3', 'Offer4', 'Agent', 'Branch', 'Call Center', 'Web', 'Four-Door Car', 'Luxury Car', 'Luxury SUV', 'SUV', 'Sports Car', 'Two-Door Car', 'Large', 'Medsize', 'Small']


,Arizona,California,Nevada,Oregon,Washington,No,Yes,Basic,Extended,Premium,...,Web,Four-Door Car,Luxury Car,Luxury SUV,SUV,Sports Car,Two-Door Car,Large,Medsize,Small
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [21]:
#Splitting into train set and test set.
#FIRST COMBINE BOTH NUMERICAL AND CATEGORICAL DATA
new_data = pd.concat([x_normalized,onehot_encoded], axis =1)
new_data.head()

,customer lifetime value,income,monthly premium auto,last_claim,months since policy inception,number of open complaints,number of policies,total claim amount,Arizona,California,...,Web,Four-Door Car,Luxury Car,Luxury SUV,SUV,Sports Car,Two-Door Car,Large,Medsize,Small
0,0.010629,0.562847,0.033755,0.914286,0.050505,0.0,0.000,0.132974,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.062406,0.000000,0.139241,0.371429,0.424242,0.0,0.875,0.391051,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.134960,0.487763,0.198312,0.514286,0.383838,0.0,0.125,0.195764,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.070589,0.000000,0.189873,0.514286,0.656566,0.0,0.750,0.183117,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.011245,0.438443,0.050633,0.342857,0.444444,0.0,0.000,0.047710,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [22]:
from sklearn.model_selection import train_test_split
#SET NEW FEATURES TO THE NEWLY CREATED DATA FRAME
X = new_data.drop(['total claim amount'], axis = 1)
y = data['total claim amount']
#NOTE: MODEL ACCURACY CHANGES TO 0.56 WHEN I USED y = new_data['total claim amount']. I expected it to be the same since i 
# did not perform any alterations on the 'total claim amount' column
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7307, 64)
(1827, 64)
(7307,)
(1827,)


In [24]:
# Modeling
#TRAIN THE MODEL
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

In [25]:
# Applying Model
from sklearn.metrics import r2_score
predictions = lm.predict(X_train)
r2_score(y_train, predictions)

0.7739829450163568

In [26]:
#EVALUATING THE MODEL USING THE TEST DATA

predictions = lm.predict(X_test)
r2_score(y_test, predictions)

0.7617818733859756

In [27]:
# Model Validation
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [28]:
mse = mean_squared_error(y_test, predictions)
print(mse)

18940.971177014042


In [29]:
rmse = math.sqrt(mse)
print(rmse)

137.6262009103428


In [30]:
mean_absolute_error(y_test, predictions)

94.80032948275863

In [31]:
# Reporting

# We have built a model which can predict values with an accuracy of 0.77